In [45]:
import pandas as pd

columns = ['kwh','interval_end','interval_width','site_id']
df = pd.read_csv("vltsdb.csv", names= columns)
df['kw'] = df['kwh']*(3600./df['interval_width'])
df

,kwh,interval_end,interval_width,site_id,kw
0,43.2,2017-07-16 02:15:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,172.8
1,42.12,2017-07-16 02:30:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,168.5
2,41.58,2017-07-16 02:45:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,166.3
3,41.22,2017-07-16 03:00:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,164.9
4,42.3,2017-07-16 03:15:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,169.2
...,...,...,...,...,...
16648407,43.74,2017-07-16 01:00:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,175
16648408,42.48,2017-07-16 01:15:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,169.9
16648409,43.2,2017-07-16 01:30:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,172.8
16648410,41.76,2017-07-16 01:45:00,900,67558069-06d2-2b8a-e0a8-18cc5444c287,167


In [11]:
import numpy as np
from pandas.tseries.frequencies import to_offset
from datetime import timedelta
from scipy.fftpack import fft, fftfreq
from statsmodels.tsa.stattools import acf
import pickle

file_name = "fifteen_minute_sampled.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

seasonality_check_subhourly_sampled = []

freq_data = '15min'

df_time_delta = pd.to_timedelta(to_offset(freq_data))

# Daily and Weekly seasons expected values from DF
time_lags_expected = [int(timedelta(days=1) / df_time_delta), int(timedelta(days=7) / df_time_delta)]

for id in loaded_list:
    # get top 3 seasons
    no_of_seasons=3
    
    df1= df.loc[df['site_id']== id]
    print(id+' ===>')
    df1.interval_end = pd.to_datetime(df1.interval_end)
    df1.index= df1.interval_end
    df1.sort_index()
    df1 = df1['kw']
    
    series = np.asarray(df1)
    # Compute FFT
    series_fft = fft(series)

    # Compute the power
    power = np.abs(series_fft)

    # Get the corresponding frequencies
    sample_freq = fftfreq(series_fft.size)

    # Find the peak frequency: we only need the positive frequencies
    pos_mask = np.where(sample_freq > 0)
    freqs = sample_freq[pos_mask]
    powers = power[pos_mask]

    # find top frequencies and corresponding time periods for seasonal pattern
    top_powers = np.argpartition(powers, -no_of_seasons)[-no_of_seasons:]

    time_periods_from_fft = 1 / freqs[top_powers]
    time_periods = time_periods_from_fft.astype(int)
    count = 0

    # One of the seasonality returned from FFT should be within range of Expected time period
    for time_lag in time_lags_expected:
        nearest_time_lag = time_periods.flat[np.abs(time_periods - time_lag).argmin()]

        #Using 5% for range comaprison
        if nearest_time_lag in range(
                time_lag - int(0.05 * time_lag),
                time_lag + int(0.05 * time_lag)):

            # Check ACF value with lags identified from FFT.
            acf_score = acf(series, nlags=nearest_time_lag)[-1]

            # Check ACF is significant or not.
            if acf_score >= 2 / np.sqrt(len(series)):
                # ACF is significant and FFT identifies seasonality
                count = count + 1
                print(str(count)+' for '+str(time_lag)+' seasonality,the site_id '+ id+' '+'is seasonal')
        else:
            print(str(count)+' for'+' '+str(time_lag)+ ' seasonality,'+'the site_id '+ id+'  Seasonality could not be identified')
    data = pd.DataFrame({"site_id":[id],"seasonality":[count]})
    seasonality_check_subhourly_sampled. append(data)

seasonality_check_subhourly_sampled  = pd.concat(seasonality_check_subhourly_sampled)
seasonality_check_subhourly_sampled 

67558069-06d2-2b8a-e0a8-18cc5444c287 ===>


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


1 for 96 seasonality,the site_id 67558069-06d2-2b8a-e0a8-18cc5444c287 is seasonal
2 for 672 seasonality,the site_id 67558069-06d2-2b8a-e0a8-18cc5444c287 is seasonal
de387a8e-521d-bb89-84a1-650ed26e5437 ===>
1 for 96 seasonality,the site_id de387a8e-521d-bb89-84a1-650ed26e5437 is seasonal
1 for 672 seasonality,the site_id de387a8e-521d-bb89-84a1-650ed26e5437  Seasonality could not be identified
25b9093b-4a6a-8c35-1545-e25cee8b7f3a ===>
1 for 96 seasonality,the site_id 25b9093b-4a6a-8c35-1545-e25cee8b7f3a is seasonal
1 for 672 seasonality,the site_id 25b9093b-4a6a-8c35-1545-e25cee8b7f3a  Seasonality could not be identified
d1668f39-1f93-e186-9d6c-acb8ba6567e4 ===>
1 for 96 seasonality,the site_id d1668f39-1f93-e186-9d6c-acb8ba6567e4 is seasonal
1 for 672 seasonality,the site_id d1668f39-1f93-e186-9d6c-acb8ba6567e4  Seasonality could not be identified
d243bd72-6f64-b597-3db6-72159081dcf4 ===>
1 for 96 seasonality,the site_id d243bd72-6f64-b597-3db6-72159081dcf4 is seasonal
1 for 672 seaso

0e5f3c13-b90f-b044-dfb9-b5192754f78a ===>
1 for 96 seasonality,the site_id 0e5f3c13-b90f-b044-dfb9-b5192754f78a is seasonal
1 for 672 seasonality,the site_id 0e5f3c13-b90f-b044-dfb9-b5192754f78a  Seasonality could not be identified
1a70c79d-b5fd-0724-0442-327519687fe6 ===>
1 for 96 seasonality,the site_id 1a70c79d-b5fd-0724-0442-327519687fe6 is seasonal
1 for 672 seasonality,the site_id 1a70c79d-b5fd-0724-0442-327519687fe6  Seasonality could not be identified
8ab0b38f-bad7-2fae-1ba6-24e2eb146f82 ===>
1 for 96 seasonality,the site_id 8ab0b38f-bad7-2fae-1ba6-24e2eb146f82 is seasonal
1 for 672 seasonality,the site_id 8ab0b38f-bad7-2fae-1ba6-24e2eb146f82  Seasonality could not be identified
8fcfa75c-e407-187d-c37c-784cac8e6c7b ===>
1 for 96 seasonality,the site_id 8fcfa75c-e407-187d-c37c-784cac8e6c7b is seasonal
1 for 672 seasonality,the site_id 8fcfa75c-e407-187d-c37c-784cac8e6c7b  Seasonality could not be identified
93833ebe-84d8-577e-7fd6-84646a06e588 ===>
1 for 96 seasonality,the site_

14c78171-fcca-bf8a-9d0b-2a75bce2b5e9 ===>
1 for 96 seasonality,the site_id 14c78171-fcca-bf8a-9d0b-2a75bce2b5e9 is seasonal
1 for 672 seasonality,the site_id 14c78171-fcca-bf8a-9d0b-2a75bce2b5e9  Seasonality could not be identified
e29e903f-f76f-1057-4fc7-4c2d91c09067 ===>
1 for 96 seasonality,the site_id e29e903f-f76f-1057-4fc7-4c2d91c09067 is seasonal
1 for 672 seasonality,the site_id e29e903f-f76f-1057-4fc7-4c2d91c09067  Seasonality could not be identified
a2c4873d-ee68-7f14-b485-10f4c400ba59 ===>
1 for 96 seasonality,the site_id a2c4873d-ee68-7f14-b485-10f4c400ba59 is seasonal
1 for 672 seasonality,the site_id a2c4873d-ee68-7f14-b485-10f4c400ba59  Seasonality could not be identified
7b59d921-c882-0ca0-7f89-fb8d60362225 ===>
1 for 96 seasonality,the site_id 7b59d921-c882-0ca0-7f89-fb8d60362225 is seasonal
1 for 672 seasonality,the site_id 7b59d921-c882-0ca0-7f89-fb8d60362225  Seasonality could not be identified
7e942ab7-170a-97da-c38f-c052a53b1b90 ===>
1 for 96 seasonality,the site_

...

7beee54b-ea35-d1d2-c01f-1869a880773a ===>
1 for 96 seasonality,the site_id 7beee54b-ea35-d1d2-c01f-1869a880773a is seasonal
1 for 672 seasonality,the site_id 7beee54b-ea35-d1d2-c01f-1869a880773a  Seasonality could not be identified
e849b316-0bd6-e691-4ebc-ce098ad8435d ===>
1 for 96 seasonality,the site_id e849b316-0bd6-e691-4ebc-ce098ad8435d is seasonal
1 for 672 seasonality,the site_id e849b316-0bd6-e691-4ebc-ce098ad8435d  Seasonality could not be identified
eadfee31-e99a-f2af-0975-37d3e9b39af3 ===>
1 for 96 seasonality,the site_id eadfee31-e99a-f2af-0975-37d3e9b39af3 is seasonal
1 for 672 seasonality,the site_id eadfee31-e99a-f2af-0975-37d3e9b39af3  Seasonality could not be identified
6f493bda-5d1e-6dac-2438-2f79d51b7c6e ===>
1 for 96 seasonality,the site_id 6f493bda-5d1e-6dac-2438-2f79d51b7c6e is seasonal
1 for 672 seasonality,the site_id 6f493bda-5d1e-6dac-2438-2f79d51b7c6e  Seasonality could not be identified
2c3dd4f7-7ab3-a0e8-4485-0e295050abd5 ===>
1 for 96 seasonality,the site_

9e5958d7-e707-bb5f-826a-2ee8c17bf297 ===>
1 for 96 seasonality,the site_id 9e5958d7-e707-bb5f-826a-2ee8c17bf297 is seasonal
1 for 672 seasonality,the site_id 9e5958d7-e707-bb5f-826a-2ee8c17bf297  Seasonality could not be identified
5961e490-81ba-c338-86cd-ad5c7919ee60 ===>
1 for 96 seasonality,the site_id 5961e490-81ba-c338-86cd-ad5c7919ee60 is seasonal
1 for 672 seasonality,the site_id 5961e490-81ba-c338-86cd-ad5c7919ee60  Seasonality could not be identified
a03f0e3d-e65c-be54-d81f-8d29058cada5 ===>
1 for 96 seasonality,the site_id a03f0e3d-e65c-be54-d81f-8d29058cada5 is seasonal
1 for 672 seasonality,the site_id a03f0e3d-e65c-be54-d81f-8d29058cada5  Seasonality could not be identified
cc7bb9d3-3ef7-979e-1603-70fa8dbcb70e ===>
1 for 96 seasonality,the site_id cc7bb9d3-3ef7-979e-1603-70fa8dbcb70e is seasonal
1 for 672 seasonality,the site_id cc7bb9d3-3ef7-979e-1603-70fa8dbcb70e  Seasonality could not be identified
5a717cb9-181c-1d40-e12f-a2777bc78952 ===>
1 for 96 seasonality,the site_

1 for 96 seasonality,the site_id b9564e11-9942-536d-c540-6a9173e63456 is seasonal
1 for 672 seasonality,the site_id b9564e11-9942-536d-c540-6a9173e63456  Seasonality could not be identified
d5e319d6-b7e6-f902-8aa7-a740ffdf3ef6 ===>
1 for 96 seasonality,the site_id d5e319d6-b7e6-f902-8aa7-a740ffdf3ef6 is seasonal
1 for 672 seasonality,the site_id d5e319d6-b7e6-f902-8aa7-a740ffdf3ef6  Seasonality could not be identified
a83e1a23-bef7-4772-76ff-b72a3e64726c ===>
1 for 96 seasonality,the site_id a83e1a23-bef7-4772-76ff-b72a3e64726c is seasonal
1 for 672 seasonality,the site_id a83e1a23-bef7-4772-76ff-b72a3e64726c  Seasonality could not be identified
81141f05-d6e9-4ea6-36b8-0bf0509137bd ===>
1 for 96 seasonality,the site_id 81141f05-d6e9-4ea6-36b8-0bf0509137bd is seasonal
1 for 672 seasonality,the site_id 81141f05-d6e9-4ea6-36b8-0bf0509137bd  Seasonality could not be identified
4a169d3c-b1ec-414c-d1ea-8736507dca6e ===>
1 for 96 seasonality,the site_id 4a169d3c-b1ec-414c-d1ea-8736507dca6e is

1 for 96 seasonality,the site_id 275e824c-e88d-8389-18a9-21e2d0e0d0f6 is seasonal
2 for 672 seasonality,the site_id 275e824c-e88d-8389-18a9-21e2d0e0d0f6 is seasonal
b6fd0560-0ae2-f650-2d0c-0d92e988af6e ===>
1 for 96 seasonality,the site_id b6fd0560-0ae2-f650-2d0c-0d92e988af6e is seasonal
2 for 672 seasonality,the site_id b6fd0560-0ae2-f650-2d0c-0d92e988af6e is seasonal
549eebeb-2468-755b-3110-c3080e17eabd ===>
1 for 96 seasonality,the site_id 549eebeb-2468-755b-3110-c3080e17eabd is seasonal
2 for 672 seasonality,the site_id 549eebeb-2468-755b-3110-c3080e17eabd is seasonal
1d07f991-8621-bbc8-d597-39c6b125f662 ===>
1 for 96 seasonality,the site_id 1d07f991-8621-bbc8-d597-39c6b125f662 is seasonal
2 for 672 seasonality,the site_id 1d07f991-8621-bbc8-d597-39c6b125f662 is seasonal
34816db2-ed2c-16d6-8416-b3849f02e9eb ===>
1 for 96 seasonality,the site_id 34816db2-ed2c-16d6-8416-b3849f02e9eb is seasonal
1 for 672 seasonality,the site_id 34816db2-ed2c-16d6-8416-b3849f02e9eb  Seasonality could 

,site_id,seasonality
0,67558069-06d2-2b8a-e0a8-18cc5444c287,2
0,de387a8e-521d-bb89-84a1-650ed26e5437,1
0,25b9093b-4a6a-8c35-1545-e25cee8b7f3a,1
0,d1668f39-1f93-e186-9d6c-acb8ba6567e4,1
0,d243bd72-6f64-b597-3db6-72159081dcf4,1
...,...,...
0,80003fc3-a5cc-a190-a51e-1861cc7ffe03,1
0,33450d85-96f7-df00-50f6-0afe4b72ab2f,2
0,5d965a76-b72f-6dc7-165f-531184c27ea9,2
0,ec60ce2e-5f91-5a8f-3106-90c84f9addcf,2


In [12]:
seasonality_check_subhourly_sampled.to_csv('fifteen_min_sampled_seasonality_check.csv')

In [8]:
import pickle
import numpy as np
from pandas.tseries.frequencies import to_offset
from datetime import timedelta
from scipy.fftpack import fft, fftfreq
from statsmodels.tsa.stattools import acf

file_name = "hourly_sampled.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

freq_data = '1H'

df_time_delta = pd.to_timedelta(to_offset(freq_data))

# Daily and Weekly seasons expected values from DF
time_lags_expected = [int(timedelta(days=1) / df_time_delta), int(timedelta(days=7) / df_time_delta)]

seasonality_check_hourly_sampled = []


for id in loaded_list:
    # get top 3 seasons
    no_of_seasons=3
    
    df1= df.loc[df['site_id']== id]
    print(id+' ===>')
    df1.interval_end = pd.to_datetime(df1.interval_end)
    df1.index= df1.interval_end
    df1.sort_index()
    df1 = df1['kw']
    
    series = np.asarray(df1)
    series_fft = fft(series)
    power = np.abs(series_fft)
    sample_freq = fftfreq(series_fft.size)
    pos_mask = np.where(sample_freq > 0)
    freqs = sample_freq[pos_mask]
    powers = power[pos_mask]
    top_powers = np.argpartition(powers, -no_of_seasons)[-no_of_seasons:]
    time_periods_from_fft = 1 / freqs[top_powers]
    time_periods = time_periods_from_fft.astype(int)
    
    count = 0
    for time_lag in time_lags_expected:
        nearest_time_lag = time_periods.flat[np.abs(time_periods - time_lag).argmin()]

        #Using 5% for range comaprison
        if nearest_time_lag in range(
                time_lag - int(0.05 * time_lag),
                time_lag + int(0.05 * time_lag)):

            # Check ACF value with lags identified from FFT.
            acf_score = acf(series, nlags=nearest_time_lag)[-1]

            # Check ACF is significant or not.
            if acf_score >= 2 / np.sqrt(len(series)):
                # ACF is significant and FFT identifies seasonality
                count = count + 1
                print(str(count)+' for '+str(time_lag)+' seasonality,the site_id '+ id+' '+'is seasonal')
        else:
            print(str(count)+' for'+' '+str(time_lag)+ ' seasonality,'+'the site_id '+ id+'  Seasonality could not be identified')
    data = pd.DataFrame({"site_id":[id],"seasonality":[count]})
    seasonality_check_hourly_sampled. append(data)

seasonality_check_hourly_sampled = pd.concat(seasonality_check_hourly_sampled)
seasonality_check_hourly_sampled 

5af67d8c-1379-a6e1-6014-5507848a20f7 ===>
0 for 24 seasonality,the site_id 5af67d8c-1379-a6e1-6014-5507848a20f7  Seasonality could not be identified
0 for 168 seasonality,the site_id 5af67d8c-1379-a6e1-6014-5507848a20f7  Seasonality could not be identified
36f7a93f-16f8-3f4d-136e-551893c0eacf ===>
0 for 168 seasonality,the site_id 36f7a93f-16f8-3f4d-136e-551893c0eacf  Seasonality could not be identified
42c0c45f-f378-fb9d-6efe-175c1b784a3a ===>
1 for 24 seasonality,the site_id 42c0c45f-f378-fb9d-6efe-175c1b784a3a is seasonal
2 for 168 seasonality,the site_id 42c0c45f-f378-fb9d-6efe-175c1b784a3a is seasonal
8d7108e3-ac36-ba4a-ad12-34b6ebe6664c ===>
1 for 24 seasonality,the site_id 8d7108e3-ac36-ba4a-ad12-34b6ebe6664c is seasonal
2 for 168 seasonality,the site_id 8d7108e3-ac36-ba4a-ad12-34b6ebe6664c is seasonal
2ae3a189-e627-5fe7-d0ac-dcbbfbab0b43 ===>
0 for 168 seasonality,the site_id 2ae3a189-e627-5fe7-d0ac-dcbbfbab0b43  Seasonality could not be identified
21e4f4d5-363b-14bd-a730-9d435

44e37900-ae24-c5ec-b82c-c1ee16f051e6 ===>
1 for 24 seasonality,the site_id 44e37900-ae24-c5ec-b82c-c1ee16f051e6 is seasonal
1 for 168 seasonality,the site_id 44e37900-ae24-c5ec-b82c-c1ee16f051e6  Seasonality could not be identified
8bc29311-3d00-1ec4-4632-9bf8c1db339e ===>
1 for 24 seasonality,the site_id 8bc29311-3d00-1ec4-4632-9bf8c1db339e is seasonal
1 for 168 seasonality,the site_id 8bc29311-3d00-1ec4-4632-9bf8c1db339e  Seasonality could not be identified
318467ad-e593-a20a-fe23-daf9998cc991 ===>
1 for 24 seasonality,the site_id 318467ad-e593-a20a-fe23-daf9998cc991 is seasonal
1 for 168 seasonality,the site_id 318467ad-e593-a20a-fe23-daf9998cc991  Seasonality could not be identified
fa1e6d9b-9562-02aa-0a4a-3f08fdf1c960 ===>
1 for 24 seasonality,the site_id fa1e6d9b-9562-02aa-0a4a-3f08fdf1c960 is seasonal
1 for 168 seasonality,the site_id fa1e6d9b-9562-02aa-0a4a-3f08fdf1c960  Seasonality could not be identified
92ee232b-db76-9552-baee-7a276d7beda5 ===>
1 for 24 seasonality,the site_

03dde450-8366-1cb8-1632-f6c8135b8077 ===>
1 for 24 seasonality,the site_id 03dde450-8366-1cb8-1632-f6c8135b8077 is seasonal
2 for 168 seasonality,the site_id 03dde450-8366-1cb8-1632-f6c8135b8077 is seasonal
35834753-fd0f-ebc7-ba6d-27c9c8de1656 ===>
1 for 24 seasonality,the site_id 35834753-fd0f-ebc7-ba6d-27c9c8de1656 is seasonal
1 for 168 seasonality,the site_id 35834753-fd0f-ebc7-ba6d-27c9c8de1656  Seasonality could not be identified
c966b497-d639-dadc-60b5-248687103a2d ===>
1 for 24 seasonality,the site_id c966b497-d639-dadc-60b5-248687103a2d is seasonal
1 for 168 seasonality,the site_id c966b497-d639-dadc-60b5-248687103a2d  Seasonality could not be identified
416a448a-266c-8622-bf29-ffb262f11dba ===>
1 for 24 seasonality,the site_id 416a448a-266c-8622-bf29-ffb262f11dba is seasonal
2 for 168 seasonality,the site_id 416a448a-266c-8622-bf29-ffb262f11dba is seasonal
d082397d-d743-fcf5-b536-762d9c7f4a35 ===>
0 for 24 seasonality,the site_id d082397d-d743-fcf5-b536-762d9c7f4a35  Seasonali

,site_id,seasonality
0,5af67d8c-1379-a6e1-6014-5507848a20f7,0
0,36f7a93f-16f8-3f4d-136e-551893c0eacf,0
0,42c0c45f-f378-fb9d-6efe-175c1b784a3a,2
0,8d7108e3-ac36-ba4a-ad12-34b6ebe6664c,2
0,2ae3a189-e627-5fe7-d0ac-dcbbfbab0b43,0
...,...,...
0,a1894b34-4d99-e549-e68c-8615418bcca0,2
0,110f08a4-548f-da24-8fd7-c6883eac9850,2
0,529d499b-a34d-324e-6c04-947e7a257a56,1
0,1c5547b1-7182-ad18-991b-0e6663379849,2


In [9]:
seasonality_check_hourly_sampled.to_csv('hourly_siteid_seasonality_result.csv')

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.4g}'.format
import enlopy as el
from sklearn.preprocessing import MaxAbsScaler
import pickle

def findFirstAndLast(arr, n, x) :
    first = -1
    last = -1
    for i in range(0, n) :
        if (x != arr[i]) :
            continue
        if (first == -1) :
            first = i
        last = i
    return [first, last+1];

file_name = "hourly_sampled.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

hourly_site_id_seasonality = pd.read_csv('hourly_siteid_seasonality_result.csv')
hourly_site_id_seasonality = hourly_site_id_seasonality[['site_id','seasonality']]


load_curve_of_hourly_seasonal_sites = []
daily_startup_shutdown_hourly_seasonal_sites = []


for i in range(0,len(loaded_list)):
    df1= df.loc[df['site_id']== loaded_list[i]]
    df2 = hourly_site_id_seasonality.loc[hourly_site_id_seasonality['site_id']==loaded_list[i]]
    print(loaded_list[i]+' ===>')
    df1.interval_end = pd.to_datetime(df1.interval_end)
    df1.index= df1.interval_end
    df1.sort_index()
    df1 = df1['kw']
    if int(df2.seasonality.values)> 0:
        load_series = el.make_timeseries(x = df1.values,  year = pd.to_datetime(df1.index[0]).year, 
                            startdate = pd.to_datetime(df1.index[0]), freq='1H') 
        try:
            load_curve  = el.analysis.get_load_archetypes(load_series, k=1, x='hour', y='dayofyear', plot_diagnostics=False)
            transformer = MaxAbsScaler().fit(load_curve)
            transformed_load_curve = transformer.transform(load_curve)
            load_curve  = load_curve.flatten()
            transformed_load_curve = transformed_load_curve.flatten()

            max_load = max(load_curve)
            on_off = []
            for j in range(0, len(transformed_load_curve)):
                if transformed_load_curve[j]>= 0.8* max_load:
                    on_off.append(1)
                else:
                    on_off.append(0)

            list_of_times = findFirstAndLast(on_off, len(on_off), 1)

            dataframe_loadcurve = pd.DataFrame({"site_id": [loaded_list[i]],"load_curve":[load_curve]}) 
            load_curve_of_hourly_seasonal_sites.append(dataframe_loadcurve)
            dataframe_of_times = pd.DataFrame({"site_id": [loaded_list[i]],"times_of_operation":[list_of_times]}) 
            daily_startup_shutdown_hourly_seasonal_sites.append(dataframe_of_times)
        
        except ValueError:
            pass

load_curve_of_hourly_seasonal_sites = pd.concat(load_curve_of_hourly_seasonal_sites)
daily_startup_shutdown_hourly_seasonal_sites = pd.concat(daily_startup_shutdown_hourly_seasonal_sites)

5af67d8c-1379-a6e1-6014-5507848a20f7 ===>


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


36f7a93f-16f8-3f4d-136e-551893c0eacf ===>
42c0c45f-f378-fb9d-6efe-175c1b784a3a ===>
8d7108e3-ac36-ba4a-ad12-34b6ebe6664c ===>


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


2ae3a189-e627-5fe7-d0ac-dcbbfbab0b43 ===>
21e4f4d5-363b-14bd-a730-9d435d74d606 ===>


...

3bb491cc-4378-1845-cd31-41b0a3c10ad3 ===>
c45693f3-d18d-5c96-246d-a669bb60f3f0 ===>


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


a1894b34-4d99-e549-e68c-8615418bcca0 ===>
110f08a4-548f-da24-8fd7-c6883eac9850 ===>
529d499b-a34d-324e-6c04-947e7a257a56 ===>
1c5547b1-7182-ad18-991b-0e6663379849 ===>


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


72f8bc16-6c01-b9f1-b89d-8c38c11f0046 ===>


In [17]:
load_curve_of_hourly_seasonal_sites.to_csv('hourly_meters_load_curve.csv')